# AMPLPY: Pattern Generation

Documentation: http://amplpy.readthedocs.io

GitHub Repository: https://github.com/ampl/amplpy

PyPI Repository: https://pypi.python.org/pypi/amplpy

### Imports

In [1]:
from __future__ import print_function
from amplpy import AMPL
import os, sys
from math import floor, ceil

In [2]:
# Import scippy
import amplpy_scip as gpy
gpy.patch(AMPL)

# if it fails to find amplpy_scip inside the virtualenv, create a kernel inside the virtualenv and use it:
# $ ipython kernel install --user --name=.venv

### Register jupyter magics for AMPL

In [3]:
from amplpy import register_magics
register_magics('_ampl_cells')  # Store %%ampl cells in the list _ampl_cells


### Basic pattern-cutting model

In [4]:
%%ampl
param nPatterns integer > 0;

set PATTERNS = 1..nPatterns;  # patterns
set WIDTHS;                   # finished widths

param order {WIDTHS} >= 0;    # rolls of width j ordered
param overrun;                # permitted overrun on any width

param rawWidth;               # width of raw rolls to be cut
param rolls {WIDTHS,PATTERNS} >= 0, default 0;   
                              # rolls of width i in pattern j

var Cut {PATTERNS} integer >= 0;  # raw rolls to cut in each pattern

minimize TotalRawRolls: sum {p in PATTERNS} Cut[p];

subject to OrderLimits {w in WIDTHS}:
  order[w] <= sum {p in PATTERNS} rolls[w,p] * Cut[p] <= order[w] + overrun;

### Plotting routine

In [5]:
def cuttingPlot(roll_width, widths, summary, solution):
    import numpy as np
    import matplotlib.pyplot as plt
    nchuncks = max(int(ceil(len(solution)/25.0)), 1)
    chunck_size = int(ceil(len(solution)/float(nchuncks)))
    start = 0
    for i in range(nchuncks):            
        plt.subplot(1, nchuncks, i+1)        
        sol = solution[start:start+chunck_size]
        start += chunck_size
        ind = np.arange(len(sol))
        acc = [0]*len(sol)
        colorlist = ['red','lightblue','orange','lightgreen',
                     'brown','fuchsia','silver','goldenrod']
        for p, (patt, rep) in enumerate(sol):
            for i in range(len(widths)):
                for j in range(patt[i]):
                    vec = [0]*len(sol)
                    vec[p] = widths[i]
                    plt.barh(ind, vec, 0.6, acc, 
                             color=colorlist[i%len(colorlist)], edgecolor='black')
                    acc[p] += widths[i]
        plt.xlim(0, roll_width)
        #plt.xticks(np.arange(0, roll_width, 10))
        plt.yticks(ind, tuple("x {:}".format(rep) for patt, rep in sol))
    plt.suptitle('{}: {} rolls, {} patterns, {:.2f}% waste'.format(
        summary['Data'],
        summary['Obj'],
        len(solution),
        round(100*summary['Waste']/(roll_width*summary['Obj']),2),
    ))   
    plt.show()

### Set parameters

In [6]:
cutdata = 'SeymaDogan.py'
#cutdata = 'ChvatalD.py'
stopdata = 'stopdata'

### Read orders, roll_width, overrun; extract widths

In [7]:
exec(open(os.path.join('CuttingCases', cutdata)).read(), globals())
widths = list(sorted(orders.keys(), reverse=True))

### Set up cutting (master problem) model

In [8]:
Master = AMPL()
Master.option['ampl_include'] = 'models'
Master.eval(_ampl_cells[0])

### Send data to AMPL

In [9]:
# Send scalar values
Master.param['nPatterns'] = len(widths)
Master.param['overrun'] = overrun
Master.param['rawWidth'] = roll_width
# Send order vector
Master.set['WIDTHS'] = widths
Master.param['order'] = orders
# Generate and send initial pattern matrix
Master.param['rolls'] = {
    (widths[i], 1+i): int(floor(roll_width/widths[i]))
    for i in range(len(widths))
}

### Set up for generation loop

In [10]:
# Define a param for sending AMPL new patterns
Master.eval('param newPat {WIDTHS} integer >= 0;')

# Set solve options
Master.option['solver'] = 'scip'
Master.option['relax_integrality'] =  1

### Define the knapsack subproblem

In [11]:
# Define knapsack subproblem
Sub = AMPL()
Sub.option['solver'] = 'scip'

Sub.eval('''
    set SIZES;
    param cap >= 0;
    param val {SIZES};
    var Qty {SIZES} integer >= 0;
    maximize TotVal: sum {s in SIZES} val[s] * Qty[s];
    subject to Cap: sum {s in SIZES} s * Qty[s] <= cap;
''')

# Send subproblem data
Sub.set['SIZES'] = widths
Sub.param['cap'] = roll_width

### Loop

In [12]:
while True:    
    Master.solve()

    Sub.param['val'].setValues(Master.con['OrderLimits'].getValues())
    Sub.solve()    
    if Sub.obj['TotVal'].value() <= 1.00001:        
        break        

    Master.param['newPat'].setValues(Sub.var['Qty'].getValues())
    Master.eval('let nPatterns := nPatterns + 1;')
    Master.eval('let {w in WIDTHS} rolls[w, nPatterns] := newPat[w];')

scip 8.1.0: optimal solution; objective 1169.000124
scip 8.1.0: optimal solution; objective 1.283333333
5 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1167.111236
scip 8.1.0: optimal solution; objective 1.283333333
4 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1165.566791
1 simplex iterations
scip 8.1.0: optimal solution; objective 1.283333333
6 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1163.677902
scip 8.1.0: optimal solution; objective 1.276923077
7 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1157.539441
4 simplex iterations
scip 8.1.0: optimal solution; objective 1.276923077
20 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1156.893287
7 simplex iterations
scip 8.1.0: optimal solution; objective 1.266666667
7 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1154.915

scip 8.1.0: optimal solution; objective 1.083028083
162 simplex iterations
93 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1042.59559
45 simplex iterations
scip 8.1.0: optimal solution; objective 1.08788289
10 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1042.555265
44 simplex iterations
scip 8.1.0: optimal solution; objective 1.087107235
3 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1040.442267
46 simplex iterations
scip 8.1.0: optimal solution; objective 1.099376924
10 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1039.709648
49 simplex iterations
scip 8.1.0: optimal solution; objective 1.08452381
57 simplex iterations
27 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1039.545122
47 simplex iterations
scip 8.1.0: optimal solution; objective 1.093257518
5 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1039.38528

scip 8.1.0: optimal solution; objective 1.004195053
3 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1027.556566
90 simplex iterations
scip 8.1.0: optimal solution; objective 1.003448228
3 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1027.461935
81 simplex iterations
scip 8.1.0: optimal solution; objective 1.003399276
3 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1027.311677
86 simplex iterations
scip 8.1.0: optimal solution; objective 1.006823314
8 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1027.305385
86 simplex iterations
scip 8.1.0: optimal solution; objective 1.00556332
10 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1027.304515
87 simplex iterations
scip 8.1.0: optimal solution; objective 1.003466141
7 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1027.253722
83

scip 8.1.0: optimal solution; objective 1026.912475
91 simplex iterations
scip 8.1.0: optimal solution; objective 1.00114515
2 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.9116
91 simplex iterations
scip 8.1.0: optimal solution; objective 1.000859122
3 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.909509
91 simplex iterations
scip 8.1.0: optimal solution; objective 1.000591514
274 simplex iterations
251 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.906022
91 simplex iterations
scip 8.1.0: optimal solution; objective 1.000481079
2 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.903921
92 simplex iterations
scip 8.1.0: optimal solution; objective 1.000413355
30 simplex iterations
19 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.90143
95 simplex iterations
scip 8.1.0: optimal solution; objective 1.000443346
24 simplex iterations


scip 8.1.0: optimal solution; objective 1.000187571
2 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.841706
99 simplex iterations
scip 8.1.0: optimal solution; objective 1.000114152
2 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.841214
97 simplex iterations
scip 8.1.0: optimal solution; objective 1.000177057
201 simplex iterations
159 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.841023
99 simplex iterations
scip 8.1.0: optimal solution; objective 1.000182493
4 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.838608
98 simplex iterations
scip 8.1.0: optimal solution; objective 1.00017785
113 simplex iterations
101 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.837637
99 simplex iterations
scip 8.1.0: optimal solution; objective 1.000130859
2 simplex iterations
1 branch-and-cut nodes
scip 8.1.0: optimal solution; objective 1026.83

### Compute and display integer solution

In [13]:
# Compute integer solution
Master.option['relax_integrality'] = 0
# Master.solve()

### Import model in scip

In [14]:
# Create scip reader and read in the NL file exported in the previous step
grb_model = Master.exportscipModel()

class MyCallback(gpy.GRBCallback):
    def run(self, where):
        try:
            if where == gpy.GRB_CB_MIP:
                runtime = self.getDouble(gpy.GRB_CB_RUNTIME)                
                if runtime >= self._stoprule['time'][self._current]:
                    print('Reducing gap tolerance to %f at %d seconds' % 
                    (self._stoprule['gaptol'][self._current], self._stoprule['time'][self._current]))
                    grb_model.setDoubleParam('MIPGap', self._stoprule['gaptol'][self._current])
                    self._current += 1                
        except Exception as e:
            print('Error:', e)
        return 0
            

# Create callback object and attach properties
cb = MyCallback()

if len(stopdata) == 0:
    cb._stoprule = {'time': (1e+10,), 'gaptol': (1,)}
else:
    exec(open(stopdata+'.py').read(), globals())
    stopdict['time'] += (1e+10,)
    stopdict['gaptol'] += (1,)
    cb._stoprule = stopdict
cb._current = 0

# Solve with scip API, using the callback defined above
grb_model.setIntParam('LogToConsole', 1)
grb_model.setCallback(cb)
grb_model.optimize()

# Export model from scip and import in the running AMPL instance
Master.importscipSolution(grb_model)

Reducing gap tolerance to 0.000200 at 15 seconds
scip 8.1.0: optimal solution; objective 1027
587587 simplex iterations
No dual variables returned.


### Retrieve solution

In [15]:
# Prepare summary data
summary = {
    'Data': cutdata,
    'Obj': int(Master.obj['TotalRawRolls'].value()),
    'Waste': Master.getValue(
             'sum {p in PATTERNS} Cut[p] * \
                (rawWidth - sum {w in WIDTHS} w*rolls[w,p])'
             )
}

# Retrieve patterns and solution
npatterns = int(Master.param['nPatterns'].value())
rolls = Master.param['rolls'].getValues().toDict()
cutvec = Master.var['Cut'].getValues().toDict()

print("%d patterns generated" % (npatterns))

# Prepare solution data
solution = [
    ([int(rolls[widths[i], p+1]) 
      for i in range(len(widths))], int(cutvec[p+1]))
    for p in range(npatterns)
    if cutvec[p+1] > 0
][:50]

print("%d patterns used" % (len(solution)))

253 patterns generated
50 patterns used


### Display solution

In [ ]:
# Create plot of solution
cuttingPlot(roll_width, widths, summary, solution)

In [17]:
Master.solve()
Master.option['scip_options'] = 'outlev=1 bestbound=1'

scip 8.1.0: optimal solution; objective 1027
88 simplex iterations


In [18]:
Master.getValue('TotalRawRolls')

1027.0